# MLP fully connected

In [1]:
%load_ext autoreload
%autoreload 2
# pretrain MLP using the new dataloader

import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["KERAS_BACKEND"] = "jax"

from typing import List
import h5py
from torch.utils.data import Dataset, DataLoader
import numpy as np
import preprocessing
from tqdm import tqdm
import keras_core as keras
from keras_core import layers
from keras_core import ops
import matplotlib.pyplot as plt

import wandb
from wandb.keras import WandbCallback, WandbMetricsLogger

from data_utils import preprocess_data, H5Dataset, H5Dataset2

Using JAX backend.


2023-10-25 01:48:23.336584: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
train_dir = "/global/ml4hep/spss/mfong/transfer_learning/delphes_train/"
train_dir_preprocess = "/global/ml4hep/spss/mfong/transfer_learning/delphes_train_processed/"
train_filepaths = [train_dir + x for x in os.listdir(train_dir)]
# preprocess_data(train_filepaths, train_dir_preprocess, force=False)
train_preprocess_file_names = os.listdir(train_dir_preprocess)
train_preprocess_filepaths = [train_dir_preprocess + name for name in train_preprocess_file_names]

In [3]:
train_preprocess_filepaths

['/global/ml4hep/spss/mfong/transfer_learning/delphes_train_processed/preprocessed_train_3.h5',
 '/global/ml4hep/spss/mfong/transfer_learning/delphes_train_processed/preprocessed_train_4.h5',
 '/global/ml4hep/spss/mfong/transfer_learning/delphes_train_processed/preprocessed_train_1.h5',
 '/global/ml4hep/spss/mfong/transfer_learning/delphes_train_processed/preprocessed_train_7.h5',
 '/global/ml4hep/spss/mfong/transfer_learning/delphes_train_processed/asdf.h5',
 '/global/ml4hep/spss/mfong/transfer_learning/delphes_train_processed/preprocessed_train_10.h5',
 '/global/ml4hep/spss/mfong/transfer_learning/delphes_train_processed/preprocessed_train_6.h5',
 '/global/ml4hep/spss/mfong/transfer_learning/delphes_train_processed/preprocessed_train_9.h5',
 '/global/ml4hep/spss/mfong/transfer_learning/delphes_train_processed/preprocessed_train_5.h5',
 '/global/ml4hep/spss/mfong/transfer_learning/delphes_train_processed/preprocessed_train_11.h5',
 '/global/ml4hep/spss/mfong/transfer_learning/delphes_

In [4]:
# total_len = 0
# for filepath in train_preprocess_filepaths:
#   with h5py.File(filepath, "r") as f:
#     total_len += len(f["labels"])
#     print("data shape =", f["data"].shape)
#     print("labels shape =", f["labels"].shape)

In [5]:
train_dataset = H5Dataset2(train_preprocess_filepaths[0:4], transform=None)
train_dataloader = DataLoader(train_dataset, batch_size=1024, shuffle=False)
print("Num train samples:", len(train_dataset))

Num train samples: 20000000


In [6]:
test_dataset = H5Dataset2(train_preprocess_filepaths[5:6], transform=None)
test_dataloader = DataLoader(test_dataset, batch_size=1024, shuffle=False)
print("Num test samples", len(test_dataset))

Num test samples 5000000


In [7]:
# next(iter(train_dataloader))[0].shape

In [8]:
config = {
  "epochs": 400,
  "batch_size": 1024,
  "learning_rate": 0.001,
  "optimizer": "adam",
  "loss": "binary_crossentropy",
  "train_samples": len(train_dataset),
  "test_samples": len(test_dataset),
}
wandb_run = wandb.init(
  project="delphes_pretrain",
  name=f"MLP_delphes",
  config=config, reinit=True
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mingfong. Use `wandb login --relogin` to force relogin


In [9]:
model = keras.Sequential([
  keras.Input(shape=(200 * 7,)),
  layers.Flatten(),
  layers.Dense(64, activation='relu'),
  layers.Dense(8, activation='relu'),
  layers.Dense(1, activation='sigmoid'),
])
model.compile(loss=config["loss"], optimizer=config["optimizer"], metrics=["accuracy", "AUC"])

2023-10-25 01:49:05.079995: W external/xla/xla/service/gpu/nvptx_compiler.cc:596] The NVIDIA driver's CUDA version is 12.0 which is older than the ptxas CUDA version (12.3.52). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [10]:
train_history = model.fit(
  train_dataloader,
  validation_data=test_dataloader,
  batch_size=config["batch_size"],
  epochs=config["epochs"],
  callbacks=[WandbMetricsLogger(log_freq="batch")],
)
# model.evaluate(test_dataloader)

Epoch 1/400


wandb: ERROR Unable to log learning rate.


   34/19532 ━━━━━━━━━━━━━━━━━━━━ 30:54:01 6s/step - accuracy: 0.6282 - auc: 0.6431 - loss: 0.9408

KeyboardInterrupt: 

In [11]:
!which python

~/anaconda3/envs/ml4hep4/bin/python


In [32]:
wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


batch/accuracy,▁▄▆▇███▇
batch/auc_2,▁▅▆▇████
batch/batch_step,▁▂▃▄▅▆▇█
batch/loss,█▅▃▃▂▂▁▁
batch/accuracy,0.62622
batch/auc_2,0.66354
batch/batch_step,7
batch/loss,0.95322
